In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Knowing the dataset

In [2]:
#Import dataset
df = pd.read_csv('base_limpa/base_unificada_limpa.csv')

C:\Users\gabri\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# Configuration to show more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
url_dicionario = 'https://docs.google.com/spreadsheets/d/1QFy_F2o81ULglNx8knNqg6oI7v3RARaUAaLLwOQr7K4/edit?usp=sharing'
f"""The dataset has {len(df)} lines and {len(df.columns)} columns. More details can be consulted in the dictionary {url_dicionario}. Below the type of variables and the 'face' from the dataset"""

In [ ]:
df.info()

In [ ]:
df.head()

#  Checking data quality

### Duplicates

In [ ]:
# Duplicate data 
duplicados = df.duplicated()
soma = duplicados.sum()
f"""The dataset has {soma} duplicate rows, which represents {round((soma/len(df)) * 100, 2)} %"""

### Missing

In [ ]:
# Counting missing values
df_sim_dofet = df[df['FLAG_BASE']=='SIM_DOFET']
missing_count_sim_dofet = df_sim_dofet.isnull().sum()  # counts the null values in each column
missing_percent_sim_dofet = round((missing_count_sim_dofet / len(df_sim_dofet)) * 100,2)  # calculates the percentage of null values
missing_data_sim_dofet = pd.DataFrame({'Missing Count': missing_count_sim_dofet, 'Missing Percentage': missing_percent_sim_dofet})
missing_data_sim_dofet.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sim_dofet.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sim_dofet['BASE'] = 'SIM_DOFET'

df_sinasc = df[df['FLAG_BASE']=='SINASC']
missing_count_sinasc = df_sinasc.isnull().sum()  # counts the null values in each column
missing_percent_sinasc = round((missing_count_sinasc / len(df_sinasc)) * 100,2)  # calculates the percentage of null values
missing_data_sinasc = pd.DataFrame({'Missing Count': missing_count_sinasc, 'Missing Percentage': missing_percent_sinasc})
missing_data_sinasc.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sinasc.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sinasc['BASE'] = 'SINASC'

# appending
missing_data = pd.concat([missing_data_sim_dofet, missing_data_sinasc])

# Sorting the DataFrame by the highest missing frequencies
missing_data_sorted = missing_data.sort_values(by=['BASE', 'Missing Count'], ascending=False)

missing_data_sorted[missing_data_sorted['Missing Percentage'] > 0]

# Some variables are not filled in the sinasc dataset. For EDA it will be used in SIM_DOFET, but in the model it will not be removed
# 'OBITOGRAV' no padding at the base
# Missing points for other variables will be removed, keeping the variables in the study

In [ ]:
del missing_data_sorted, missing_data, df_sim_dofet, df_sinasc

### Categorical variables

In [ ]:
lista_cat = [
'ano_evento'
, 'TIPOBITO'       
, 'evento_MUNNOMEX'
, 'res_MUNNOMEX'
, 'evento_CAPITAL'
, 'res_CAPITAL'
, 'evento_REGIAO'
, 'res_REGIAO'
, 'evento_SIGLA_UF'
, 'res_SIGLA_UF'
, 'idademae_faixa'
, 'ESCMAE2010'
, 'escolaridade_mae'
, 'OBITOGRAV'
, 'GRAVIDEZ'
, 'tipo_gravidez'
, 'idade_gestacao_faixa'
, 'SEXO'
, 'def_sexo'
, 'peso_faixa'
, 'OBITOPARTO'
, 'def_obito_parto'
, 'CAUSABAS'
, 'causabas_capitulo'
, 'causabas_categoria'
, 'causabas_grupo'
, 'causabas_subcategoria'
, 'FLAG_BASE'
            ]
for col in lista_cat:
    print(f'\nPercentual de valores únicos para {col}:')
    print(round((df[col].value_counts()/len(df)) * 100, 2))

### Numerical variables

In [ ]:
# Basic Statistics for Numeric Variables
lista_numerica = [
'IDADEMAE'
, 'SEMAGESTAC'
, 'PESO'
, 'sum_CENTROBS'
, 'sum_QTINST34'
, 'sum_QTINST35'
, 'sum_QTINST36'
, 'sum_QTINST37'
, 'sum_QTLEIT34'
, 'sum_QTLEIT38'
, 'sum_QTLEIT39'
, 'sum_QTLEIT40'
, 'sum_CENTRNEO'
, 'TP_UNID_5'
, 'TP_UNID_7'
, 'TP_UNID_15'
, 'TP_UNID_36'
, 'TP_UNID_61'
]
estatisticas_numericas = df[lista_numerica].describe()
estatisticas_numericas

### Outlier

In [ ]:
def detectar_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = col[(col < lower_bound) | (col > upper_bound)]
    return outliers

outliers_numericos = df[lista_numerica].apply(detectar_outliers)

for i in lista_numerica:
    print(outliers_numericos.loc[~outliers_numericos[i].isna(), [i]].describe())

In [ ]:
for i in lista_numerica:
    x1 = df.loc[df['ano_evento'] == 2019, i]
    x2 = df.loc[df['ano_evento'] == 2020, i]
    x3 = df.loc[df['ano_evento'] == 2021, i]
    x4 = df.loc[df['ano_evento'] == 2022, i]

    # Normalize
    kwargs = dict(alpha=0.5, bins=100, density=True, stacked=True)

    # Plot
    plt.figure()  # Creates a new figure for each iteration
    plt.hist(x1, **kwargs, color='g', label='2019')
    plt.hist(x2, **kwargs, color='b', label='2020')
    plt.hist(x3, **kwargs, color='r', label='2021')
    plt.hist(x4, **kwargs, color='y', label='2022')
    plt.gca().set(title=f'Histograma {i}', ylabel='Probability')
    # plt.xlim(50, 75)
    plt.legend()

    # Show or save the figure here if needed
    # plt.savefig(f'histograma_{i}.png')

    plt.show()  # Show the graph

# Some questions

#### What is the percentage of deaths by cause each year?

In [ ]:
df_sim_dofet = df.loc[df['FLAG_BASE']=='SIM_DOFET']
contingency_table = pd.crosstab(index=df_sim_dofet['causabas_categoria'], columns=df_sim_dofet['ano_evento'], normalize='columns')
contingency_table = contingency_table.reset_index()
contingency_table.sort_values(by=2019, ascending=False)

#### What is the proportion of deaths and births for each UF where the event (death or birth) occurred?

In [ ]:
df_agrupado = df.groupby(
 [
    'evento_SIGLA_UF'
    , 'FLAG_BASE'
    , 'ano_evento'
 ]
    , as_index=False
).agg(
    count_evento=pd.NamedAgg(column='evento_SIGLA_UF', aggfunc='size')
)

df_pivot = pd.pivot_table(
    df_agrupado
    , index=['evento_SIGLA_UF']
    , columns=['FLAG_BASE', 'ano_evento']
).reset_index()
df_pivot.columns = ['SIGLA_UF', 'SIM_DOFET_2019', 'SIM_DOFET_2020', 'SIM_DOFET_2021', 'SINASC_2019', 'SINASC_2020',
                     'SINASC_2021', 'SINASC_2022']

df_pivot['PROP_2019'] = df_pivot['SIM_DOFET_2019']/df_pivot['SINASC_2019']
df_pivot['PROP_2020'] = df_pivot['SIM_DOFET_2020']/df_pivot['SINASC_2020']
df_pivot['PROP_2021'] = df_pivot['SIM_DOFET_2021']/df_pivot['SINASC_2021']
df_pivot

#### Has any municipality seen a decrease in the number of beds dedicated to childbirth care?

In [10]:
df_agrupado = df.groupby(
 [
    'evento_SIGLA_UF'
    , 'evento_MUNNOMEX'
    , 'ano_evento'
 ]
    , as_index=False
).agg(
    QTINST34=pd.NamedAgg(column='sum_QTINST34', aggfunc='max')
    , QTINST35=pd.NamedAgg(column='sum_QTINST35', aggfunc='max')
    , QTINST36=pd.NamedAgg(column='sum_QTINST36', aggfunc='max')
    , QTINST37=pd.NamedAgg(column='sum_QTINST37', aggfunc='max')
    , QTLEIT34=pd.NamedAgg(column='sum_QTLEIT34', aggfunc='max')
    , QTLEIT38=pd.NamedAgg(column='sum_QTLEIT38', aggfunc='max')
    , QTLEIT39=pd.NamedAgg(column='sum_QTLEIT39', aggfunc='max')
    , QTLEIT40=pd.NamedAgg(column='sum_QTLEIT40', aggfunc='max')
    )

df_agrupado['QTINST'] = df_agrupado['QTINST34'] + df_agrupado['QTINST35'] + df_agrupado['QTINST36'] + df_agrupado['QTINST37']
df_agrupado['QTLEIT'] = df_agrupado['QTLEIT34'] + df_agrupado['QTLEIT38'] + df_agrupado['QTLEIT39'] + df_agrupado['QTLEIT40']

df_pivot = pd.pivot_table(
    df_agrupado
    , index=['evento_SIGLA_UF', 'evento_MUNNOMEX']
    , columns=['ano_evento']
    , values = ['QTINST', 'QTLEIT']
    , fill_value=0
).reset_index()
df_pivot.columns = ['evento_SIGLA_UF', 'evento_MUNNOMEX', 'QTINST_2019', 'QTINST_2020', 'QTINST_2021', 'QTINST_2022',
                   'QTLEIT_2019', 'QTLEIT_2020', 'QTLEIT_2021', 'QTLEIT_2022']

df_pivot['dif_qtinst_20_19'] = df_pivot['QTINST_2020'] - df_pivot['QTINST_2019'] 
df_pivot['dif_qtinst_21_20'] = df_pivot['QTINST_2021'] - df_pivot['QTINST_2020'] 
df_pivot['dif_qtinst_22_21'] = df_pivot['QTINST_2022'] - df_pivot['QTINST_2021'] 
df_pivot['dif_qtleit_20_19'] = df_pivot['QTLEIT_2020'] - df_pivot['QTLEIT_2019'] 
df_pivot['dif_qtleit_21_20'] = df_pivot['QTLEIT_2021'] - df_pivot['QTLEIT_2020'] 
df_pivot['dif_qtleit_22_21'] = df_pivot['QTLEIT_2022'] - df_pivot['QTLEIT_2021'] 

In [12]:
aa = df_pivot.loc[(df_pivot['dif_qtinst_20_19']<0) | (df_pivot['dif_qtinst_21_20']<0) | (df_pivot['dif_qtinst_22_21']<0) |
            (df_pivot['dif_qtleit_20_19']<0) | (df_pivot['dif_qtleit_21_20']<0) | (df_pivot['dif_qtleit_22_21']<0)]
aa
# 64% of the municipalities had some change in the years of the study

,evento_SIGLA_UF,evento_MUNNOMEX,QTINST_2019,QTINST_2020,QTINST_2021,QTINST_2022,QTLEIT_2019,QTLEIT_2020,QTLEIT_2021,QTLEIT_2022,dif_qtinst_20_19,dif_qtinst_21_20,dif_qtinst_22_21,dif_qtleit_20_19,dif_qtleit_21_20,dif_qtleit_22_21
0,AC,ACRELANDIA,30,2,23,6,196,1,197,25,-28,21,-17,-195,196,-172
1,AC,ASSIS BRASIL,2,2,17,2,0,0,53,0,0,15,-15,0,53,-53
2,AC,BRASILEIA,17,17,9,13,53,66,8,26,0,-8,4,13,-58,18
3,AC,BUJARI,0,4,0,0,0,7,0,0,4,-4,0,7,-7,0
5,AC,CRUZEIRO DO SUL,70,75,29,23,464,508,197,67,5,-46,-6,44,-311,-130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4509,TO,SILVANOPOLIS,2,2,2,4,4,14,4,8,0,0,2,10,-10,4
4511,TO,TAGUATINGA,2,2,2,2,14,14,14,5,0,0,0,0,0,-9
4514,TO,TOCANTINOPOLIS,13,90,4,5,10,168,3,6,77,-86,1,158,-165,3
4515,TO,WANDERLANDIA,2,13,2,1,2,9,2,0,11,-11,-1,7,-7,-2


#### How many municipalities had less than 12 births in a year?

# Cleaning the dataset

In [ ]:
# Removing columns with a high frequency of missings

#df_limpo = df_limpo.drop(columns=['OBITOGRAV', 'causabas_categoria', 'TIPOBITO', 'causabas_capitulo', 'CAUSABAS'
#                                  , 'def_obito_parto', 'OBITOPARTO', 'causabas_subcategoria', 'causabas_grupo'])

In [ ]:
# Removing duplicates
tam_inicial = len(df)
df_limpo = df.drop_duplicates()
tam_depois=len(df_limpo)
print(f'Remove duplicates {tam_inicial - tam_depois} rows')

In [ ]:
df_limpo = df_limpo.reset_index(drop=True)

In [ ]:
df_limpo.info()

In [ ]:
df_limpo.to_csv('base_limpa/base_unificada_modelo.csv', index=False)